# 🔧 Hyperparameter Tuning - Ensemble Models

Este notebook optimiza los hiperparámetros de los modelos ensemble (Random Forest, Gradient Boosting, AdaBoost) para maximizar su performance.

**Estrategia:** Solo optimizamos ensemble models porque ya demostraron ser superiores a baseline.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from configure import DATA_DIR, MODELS_DIR
from src.load_data import load_data

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report, confusion_matrix

import pickle
from datetime import datetime

# Configuración de visualización
plt.style.use('default')
sns.set_palette('husl')

## 📥 Cargar Datos

In [2]:
# Cargar dataset limpio
processed_path = DATA_DIR / 'processed' / 'fetal_health_clean.csv'
df = pd.read_csv(processed_path)
print(f"Dataset cargado: {df.shape}")

Dataset cargado: (2113, 22)


In [3]:
# Separar features y target
X = df.drop('fetal_health', axis=1)
y = df['fetal_health']

In [4]:
# Train/Test split (mismo que baseline y ensemble para comparación justa)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=25, stratify=y
)
print(f"Train: {X_train.shape} | Test: {X_test.shape}")

Train: (1690, 21) | Test: (423, 21)


## 📊 Cargar Resultados Base (para comparación)

In [5]:
# Cargar resultados de ensemble models SIN optimizar
ensemble_base_df = pd.read_csv(DATA_DIR / 'processed' / 'ensemble_results.csv')
print("🔹 ENSEMBLE MODELS (Base - Sin Optimizar)")
print("=" * 80)
print(ensemble_base_df.sort_values(by='Test Score', ascending=False))
print("\n")

🔹 ENSEMBLE MODELS (Base - Sin Optimizar)
            Model  Train Score  Test Score  F1 Score  Recall Score
0   Random Forest       0.9994      0.9220    0.9198        0.9220
1  Gradient Boost       0.9941      0.9196    0.9183        0.9196
2        AdaBoost       0.9095      0.8676    0.8648        0.8676




## 🎯 1. Random Forest - Hyperparameter Tuning

In [6]:
# Definir param grid para Random Forest
rf_param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, 30, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['sqrt', 'log2'],
}

# Pipeline
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=25))
])

# GridSearchCV con Cross-Validation
print("🔍 Buscando mejores hiperparámetros para Random Forest...")
print(f"   Total combinaciones: {np.prod([len(v) for v in rf_param_grid.values()])}")
print(f"   Esto puede tomar varios minutos...\n")

rf_grid = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=rf_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Entrenar
start_time = datetime.now()
rf_grid.fit(X_train, y_train)
end_time = datetime.now()

print(f"\n✅ Random Forest tuning completado en {(end_time - start_time).seconds} segundos")
print(f"🏆 Mejores parámetros: {rf_grid.best_params_}")
print(f"📊 Mejor CV Score: {rf_grid.best_score_:.4f}")

🔍 Buscando mejores hiperparámetros para Random Forest...
   Total combinaciones: 216
   Esto puede tomar varios minutos...

Fitting 5 folds for each of 216 candidates, totalling 1080 fits

✅ Random Forest tuning completado en 187 segundos
🏆 Mejores parámetros: {'classifier__max_depth': 20, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 300}
📊 Mejor CV Score: 0.9438

✅ Random Forest tuning completado en 187 segundos
🏆 Mejores parámetros: {'classifier__max_depth': 20, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 300}
📊 Mejor CV Score: 0.9438


In [7]:
# Evaluar Random Forest optimizado
rf_best = rf_grid.best_estimator_
rf_train_score = rf_best.score(X_train, y_train)
rf_test_score = rf_best.score(X_test, y_test)
rf_y_pred = rf_best.predict(X_test)
rf_f1 = f1_score(y_test, rf_y_pred, average='weighted')
rf_recall = recall_score(y_test, rf_y_pred, average='weighted')

print("🔹 RANDOM FOREST OPTIMIZADO")
print("=" * 60)
print(f"Train Score: {rf_train_score:.4f}")
print(f"Test Score:  {rf_test_score:.4f}")
print(f"F1 Score:    {rf_f1:.4f}")
print(f"Recall:      {rf_recall:.4f}")
print("\n")

🔹 RANDOM FOREST OPTIMIZADO
Train Score: 0.9893
Test Score:  0.9362
F1 Score:    0.9353
Recall:      0.9362




## 🎯 2. Gradient Boosting - Hyperparameter Tuning

In [ ]:
# Definir param grid para Gradient Boosting
gb_param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'classifier__max_depth': [3, 5, 7],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__subsample': [0.8, 0.9, 1.0],
}

# Pipeline
gb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', GradientBoostingClassifier(random_state=25))
])

# GridSearchCV con Cross-Validation
print("🔍 Buscando mejores hiperparámetros para Gradient Boosting...")
print(f"   Total combinaciones: {np.prod([len(v) for v in gb_param_grid.values()])}")
print(f"   Esto puede tomar varios minutos...\n")

gb_grid = GridSearchCV(
    estimator=gb_pipeline,
    param_grid=gb_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Entrenar
start_time = datetime.now()
gb_grid.fit(X_train, y_train)
end_time = datetime.now()

print(f"\n✅ Gradient Boosting tuning completado en {(end_time - start_time).seconds} segundos")
print(f"🏆 Mejores parámetros: {gb_grid.best_params_}")
print(f"📊 Mejor CV Score: {gb_grid.best_score_:.4f}")

🔍 Buscando mejores hiperparámetros para Gradient Boosting...
   Total combinaciones: 972
   Esto puede tomar varios minutos...

Fitting 5 folds for each of 972 candidates, totalling 4860 fits


In [ ]:
# Evaluar Gradient Boosting optimizado
gb_best = gb_grid.best_estimator_
gb_train_score = gb_best.score(X_train, y_train)
gb_test_score = gb_best.score(X_test, y_test)
gb_y_pred = gb_best.predict(X_test)
gb_f1 = f1_score(y_test, gb_y_pred, average='weighted')
gb_recall = recall_score(y_test, gb_y_pred, average='weighted')

print("🔹 GRADIENT BOOSTING OPTIMIZADO")
print("=" * 60)
print(f"Train Score: {gb_train_score:.4f}")
print(f"Test Score:  {gb_test_score:.4f}")
print(f"F1 Score:    {gb_f1:.4f}")
print(f"Recall:      {gb_recall:.4f}")
print("\n")

## 🎯 3. AdaBoost - Hyperparameter Tuning

In [ ]:
# Definir param grid para AdaBoost
ada_param_grid = {
    'classifier__n_estimators': [50, 100, 200, 300],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0],
    'classifier__algorithm': ['SAMME', 'SAMME.R'],
}

# Pipeline
ada_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', AdaBoostClassifier(random_state=25))
])

# GridSearchCV con Cross-Validation
print("🔍 Buscando mejores hiperparámetros para AdaBoost...")
print(f"   Total combinaciones: {np.prod([len(v) for v in ada_param_grid.values()])}")
print(f"   Esto puede tomar varios minutos...\n")

ada_grid = GridSearchCV(
    estimator=ada_pipeline,
    param_grid=ada_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Entrenar
start_time = datetime.now()
ada_grid.fit(X_train, y_train)
end_time = datetime.now()

print(f"\n✅ AdaBoost tuning completado en {(end_time - start_time).seconds} segundos")
print(f"🏆 Mejores parámetros: {ada_grid.best_params_}")
print(f"📊 Mejor CV Score: {ada_grid.best_score_:.4f}")

In [ ]:
# Evaluar AdaBoost optimizado
ada_best = ada_grid.best_estimator_
ada_train_score = ada_best.score(X_train, y_train)
ada_test_score = ada_best.score(X_test, y_test)
ada_y_pred = ada_best.predict(X_test)
ada_f1 = f1_score(y_test, ada_y_pred, average='weighted')
ada_recall = recall_score(y_test, ada_y_pred, average='weighted')

print("🔹 ADABOOST OPTIMIZADO")
print("=" * 60)
print(f"Train Score: {ada_train_score:.4f}")
print(f"Test Score:  {ada_test_score:.4f}")
print(f"F1 Score:    {ada_f1:.4f}")
print(f"Recall:      {ada_recall:.4f}")
print("\n")

## 📊 Comparación: Base vs Optimizado

In [ ]:
# Crear DataFrame con resultados optimizados
optimized_results = [
    ['Random Forest', rf_train_score, rf_test_score, rf_f1, rf_recall],
    ['Gradient Boosting', gb_train_score, gb_test_score, gb_f1, gb_recall],
    ['AdaBoost', ada_train_score, ada_test_score, ada_f1, ada_recall]
]

optimized_df = pd.DataFrame(
    optimized_results,
    columns=['Model', 'Train Score', 'Test Score', 'F1 Score', 'Recall Score']
)
optimized_df = optimized_df.round(4)

print("🚀 ENSEMBLE MODELS OPTIMIZADOS")
print("=" * 80)
print(optimized_df.sort_values(by='Test Score', ascending=False))
print("\n")

In [ ]:
# Comparación lado a lado
comparison_data = []

for model_name in ['Random Forest', 'Gradient Boosting', 'AdaBoost']:
    base_score = ensemble_base_df[ensemble_base_df['Model'] == model_name]['Test Score'].values[0]
    opt_score = optimized_df[optimized_df['Model'] == model_name]['Test Score'].values[0]
    improvement_abs = opt_score - base_score
    improvement_rel = (improvement_abs / base_score) * 100
    
    comparison_data.append([
        model_name,
        base_score,
        opt_score,
        improvement_abs,
        improvement_rel
    ])

comparison_df = pd.DataFrame(
    comparison_data,
    columns=['Model', 'Base Score', 'Optimized Score', 'Improvement (Abs)', 'Improvement (%)']
)
comparison_df = comparison_df.round(4)

print("📈 COMPARACIÓN: BASE vs OPTIMIZADO")
print("=" * 80)
print(comparison_df.sort_values(by='Improvement (%)', ascending=False))
print("\n")

## 📊 Visualización Comparativa

In [ ]:
# Gráfico de comparación Base vs Optimizado
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Scores Base vs Optimizado
x = np.arange(len(comparison_df))
width = 0.35

ax1 = axes[0]
bars1 = ax1.bar(x - width/2, comparison_df['Base Score'], width, label='Base', alpha=0.8, color='#3498db')
bars2 = ax1.bar(x + width/2, comparison_df['Optimized Score'], width, label='Optimized', alpha=0.8, color='#e74c3c')

ax1.set_xlabel('Model', fontsize=12, fontweight='bold')
ax1.set_ylabel('Test Score', fontsize=12, fontweight='bold')
ax1.set_title('Base vs Optimized - Test Scores', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(comparison_df['Model'], rotation=15, ha='right')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.set_ylim([0.85, 1.0])

# Añadir valores en las barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

# Plot 2: Mejora porcentual
ax2 = axes[1]
bars3 = ax2.bar(comparison_df['Model'], comparison_df['Improvement (%)'], 
                alpha=0.8, color='#2ecc71', edgecolor='black')

ax2.set_xlabel('Model', fontsize=12, fontweight='bold')
ax2.set_ylabel('Improvement (%)', fontsize=12, fontweight='bold')
ax2.set_title('Mejora Relativa después del Tuning', fontsize=14, fontweight='bold')
ax2.set_xticklabels(comparison_df['Model'], rotation=15, ha='right')
ax2.grid(axis='y', alpha=0.3, linestyle='--')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.8)

# Añadir valores en las barras
for bar in bars3:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
            f'{height:.2f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## 🏆 Identificar Mejor Modelo Optimizado

In [ ]:
# Identificar el mejor modelo optimizado
best_idx = optimized_df['Test Score'].idxmax()
best_model_name = optimized_df.loc[best_idx, 'Model']
best_test_score = optimized_df.loc[best_idx, 'Test Score']
best_f1 = optimized_df.loc[best_idx, 'F1 Score']
best_recall = optimized_df.loc[best_idx, 'Recall Score']

# Obtener el modelo correspondiente
if best_model_name == 'Random Forest':
    best_model = rf_best
    best_params = rf_grid.best_params_
elif best_model_name == 'Gradient Boosting':
    best_model = gb_best
    best_params = gb_grid.best_params_
else:  # AdaBoost
    best_model = ada_best
    best_params = ada_grid.best_params_

print("="*80)
print("🏆 MEJOR MODELO OPTIMIZADO")
print("="*80)
print(f"Modelo:       {best_model_name}")
print(f"Test Score:   {best_test_score:.4f}")
print(f"F1 Score:     {best_f1:.4f}")
print(f"Recall Score: {best_recall:.4f}")
print("\nMejores Parámetros:")
for param, value in best_params.items():
    print(f"  - {param}: {value}")
print("="*80)

## 🎯 Matriz de Confusión del Mejor Modelo

In [ ]:
# Matriz de confusión del mejor modelo optimizado
y_pred_best = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Suspect', 'Pathological'], 
            yticklabels=['Normal', 'Suspect', 'Pathological'],
            cbar_kws={'label': 'Count'})
plt.ylabel('Actual', fontsize=13, fontweight='bold')
plt.xlabel('Predicted', fontsize=13, fontweight='bold')
plt.title(f'Confusion Matrix - {best_model_name} (Optimized)', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

## 📋 Classification Report

In [ ]:
# Classification report detallado
print("📊 CLASSIFICATION REPORT - MEJOR MODELO")
print("=" * 80)
print(classification_report(y_test, y_pred_best, 
                          target_names=['Normal', 'Suspect', 'Pathological']))

## 💾 Guardar Resultados y Mejor Modelo

In [ ]:
# Guardar resultados optimizados
optimized_df.to_csv(DATA_DIR / 'processed' / 'optimized_results.csv', index=False)
print(f"✅ Resultados optimizados guardados en: {DATA_DIR / 'processed' / 'optimized_results.csv'}")

# Guardar comparación
comparison_df.to_csv(DATA_DIR / 'processed' / 'tuning_comparison.csv', index=False)
print(f"✅ Comparación guardada en: {DATA_DIR / 'processed' / 'tuning_comparison.csv'}")

In [ ]:
# Guardar mejor modelo optimizado
model_filename = MODELS_DIR / f'best_model_{best_model_name.replace(" ", "_").lower()}_optimized.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

print(f"✅ Mejor modelo guardado en: {model_filename}")

# Guardar metadata del mejor modelo
best_model_metadata = {
    'model_name': best_model_name,
    'test_score': best_test_score,
    'f1_score': best_f1,
    'recall_score': best_recall,
    'best_params': best_params,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

metadata_filename = MODELS_DIR / 'best_model_metadata.pkl'
with open(metadata_filename, 'wb') as f:
    pickle.dump(best_model_metadata, f)

print(f"✅ Metadata guardada en: {metadata_filename}")

## 📋 Resumen Final del Tuning

In [ ]:
print("="*80)
print("📊 RESUMEN FINAL - HYPERPARAMETER TUNING")
print("="*80)
print("\n🎯 Modelos Optimizados:")
for _, row in comparison_df.iterrows():
    print(f"\n  {row['Model']}:")
    print(f"    Base Score:      {row['Base Score']:.4f}")
    print(f"    Optimized Score: {row['Optimized Score']:.4f}")
    print(f"    Mejora:          {row['Improvement (Abs)']:.4f} ({row['Improvement (%)']:.2f}%)")

print(f"\n\n🏆 MODELO FINAL SELECCIONADO:")
print(f"   {best_model_name}")
print(f"   Test Score: {best_test_score:.4f}")
print(f"   F1 Score:   {best_f1:.4f}")
print(f"\n📁 Archivos generados:")
print(f"   - {DATA_DIR / 'processed' / 'optimized_results.csv'}")
print(f"   - {DATA_DIR / 'processed' / 'tuning_comparison.csv'}")
print(f"   - {model_filename}")
print(f"   - {metadata_filename}")
print("="*80)

## 🎯 Conclusiones

### Resultados del Tuning:
- **Modelos optimizados:** Random Forest, Gradient Boosting, AdaBoost
- **Mejor modelo:** [Se completa automáticamente al ejecutar]
- **Mejora obtenida:** [Se completa automáticamente al ejecutar]

### Observaciones:
- GridSearchCV con 5-fold CV garantiza robustez
- Los parámetros óptimos varían según el modelo
- Gradient Boosting suele mostrar mayor mejora con tuning
- Random Forest es más robusto con parámetros por defecto

### Próximos Pasos:
- ✅ Modelos ensemble optimizados
- ✅ Mejor modelo identificado y guardado
- ⏭️ Evaluación profunda con learning curves
- ⏭️ Feature importance analysis
- ⏭️ Pipeline de deployment/inference